In [2]:
import pandas as pd

# Шлях до завантаженого файлу
file_path = "data2.csv"

# Завантаження даних
df = pd.read_csv(file_path)

# Виведення основної інформації про датасет
df_info = df.info()
df_head = df.head()

df_info, df_head

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 33 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       569 non-null    int64  
 1   diagnosis                569 non-null    object 
 2   radius_mean              569 non-null    float64
 3   texture_mean             569 non-null    float64
 4   perimeter_mean           569 non-null    float64
 5   area_mean                569 non-null    float64
 6   smoothness_mean          569 non-null    float64
 7   compactness_mean         569 non-null    float64
 8   concavity_mean           569 non-null    float64
 9   concave points_mean      569 non-null    float64
 10  symmetry_mean            569 non-null    float64
 11  fractal_dimension_mean   569 non-null    float64
 12  radius_se                569 non-null    float64
 13  texture_se               569 non-null    float64
 14  perimeter_se             5

(None,
          id diagnosis  radius_mean  texture_mean  perimeter_mean  area_mean  \
 0    842302         M        17.99         10.38          122.80     1001.0   
 1    842517         M        20.57         17.77          132.90     1326.0   
 2  84300903         M        19.69         21.25          130.00     1203.0   
 3  84348301         M        11.42         20.38           77.58      386.1   
 4  84358402         M        20.29         14.34          135.10     1297.0   
 
    smoothness_mean  compactness_mean  concavity_mean  concave points_mean  \
 0          0.11840           0.27760          0.3001              0.14710   
 1          0.08474           0.07864          0.0869              0.07017   
 2          0.10960           0.15990          0.1974              0.12790   
 3          0.14250           0.28390          0.2414              0.10520   
 4          0.10030           0.13280          0.1980              0.10430   
 
    ...  texture_worst  perimeter_worst  

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

# Видаляємо некорисні стовпці
df = df.drop(columns=["id", "Unnamed: 32"])

# Закодуємо цільову змінну (M=1, B=0)
df["diagnosis"] = LabelEncoder().fit_transform(df["diagnosis"])

# Розділяємо на ознаки (X) і цільову змінну (y)
X = df.drop(columns=["diagnosis"])
y = df["diagnosis"]

# Масштабуємо ознаки для коректної роботи регуляризації
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Розділяємо дані на навчальний і тестовий набори (80% - train, 20% - test)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Перевіряємо розміри вибірок
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((455, 30), (114, 30), (455,), (114,))

In [4]:
from sklearn.linear_model import Ridge, Lasso
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Навчання Ridge регресії (L2 регуляризація)
ridge = Ridge(alpha=1.0)  # Гіперпараметр α можна змінювати
ridge.fit(X_train, y_train)
y_pred_ridge = ridge.predict(X_test)

# Навчання Lasso регресії (L1 регуляризація)
lasso = Lasso(alpha=0.1)  # Гіперпараметр α можна змінювати
lasso.fit(X_train, y_train)
y_pred_lasso = lasso.predict(X_test)

# Оцінка точності моделей
ridge_mse = mean_squared_error(y_test, y_pred_ridge)
ridge_mae = mean_absolute_error(y_test, y_pred_ridge)

lasso_mse = mean_squared_error(y_test, y_pred_lasso)
lasso_mae = mean_absolute_error(y_test, y_pred_lasso)

ridge_mse, ridge_mae, lasso_mse, lasso_mae

(0.06205316603891845,
 0.1958184407686583,
 0.07549697688418236,
 0.23390920229170312)

In [5]:
import numpy as np

# Отримуємо коефіцієнти моделей
ridge_coeffs = ridge.coef_
lasso_coeffs = lasso.coef_

# Формуємо таблицю коефіцієнтів
coeff_df = pd.DataFrame({
    "Feature": X.columns,
    "Ridge Coeff": ridge_coeffs,
    "Lasso Coeff": lasso_coeffs
})

# Визначаємо значущі ознаки (ті, що не занулені в Lasso)
important_features = coeff_df[coeff_df["Lasso Coeff"] != 0]

# Виводимо топ-10 найбільш значущих ознак для Lasso
coeff_df_sorted = important_features.reindex(important_features["Lasso Coeff"].abs().sort_values(ascending=False).index)
coeff_df_sorted.head(10)

,Feature,Ridge Coeff,Lasso Coeff
27,concave points_worst,-0.044671,0.158643
20,radius_worst,0.375227,0.122368
7,concave points_mean,0.167684,0.025786
21,texture_worst,0.065117,0.014775
